In [1]:
import numpy as np
import sys; sys.path.insert(0, '../')
from gaia_tools.query import make_query, make_simple_query
import warnings; warnings.filterwarnings('ignore')

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


# Simple Query


This is a simple single-level query.



## Default Columns

In [2]:
circle = """
--Selections: Cluster RA
1=CONTAINS(POINT('ICRS',gaia.ra,gaia.dec),
           CIRCLE('ICRS',{ra:.4f},{dec:.4f},{rad:.2f}))
""".format(ra=230, dec=0, rad=4)

In [3]:
df = make_simple_query(
    WHERE=circle,        # The WHERE part of the SQL
    random_index=1e4,         # a shortcut to use the random_index in 'WHERE'
    ORDERBY='gaia.parallax',  # setting the data ordering
    pprint=True,              # print the query
    do_query=True,            # perform the query using gaia_tools.query
    local=False,              # whether to perform the query locally
    units=True                # to fill in missing units from 'defaults' file
)

df

--Data Columns:
SELECT
--GaiaDR2 Columns:
gaia.source_id AS id,
gaia.parallax AS prlx, gaia.parallax_error AS prlx_err,
gaia.ra, gaia.ra_error AS ra_err,
gaia.dec, gaia.dec_error AS dec_err,
gaia.pmra, gaia.pmra_error AS pmra_err,
gaia.pmdec, gaia.pmdec_error AS pmdec_err


--SOURCE:
FROM gaiadr2.gaia_source AS gaia

--Selections:
WHERE
--Selections: Cluster RA
1=CONTAINS(POINT('ICRS',gaia.ra,gaia.dec),
           CIRCLE('ICRS',230.0000,0.0000,4.00))

AND random_index < 10000

--Ordering:
ORDER BY
gaia.parallax


starting query @ m04d02h18m24s59
Query finished.
query finished @ m04d02h18m25s00


id,prlx,prlx_err,ra,ra_err,dec,dec_err,pmra,pmra_err,pmdec,pmdec_err
,mas,mas,deg,deg,deg,deg,mas / yr,mas / yr,mas / yr,mas / yr
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
4421854511258656512,1.2275574478067548,0.47971770270355213,228.8875273212023,0.47414322708214046,2.6766887835445843,0.4727195334587449,-1.086205413366316,0.8596805506822935,8.1358729037405,0.8617703124298011
4421967451718550400,1.3590008052478864,0.41067570449898166,229.30428207922225,0.32917441643105966,3.504297860472648,0.30141853305113375,-15.617658856828964,0.69020127163748,-11.519501862147981,0.6119762121388062
6336481490785920000,1.9709522874255738,1.4830800617599345,227.8835097013005,1.527311600754504,-3.008821326198841,1.2609050214395636,-18.116198677523734,4.053641595094551,-5.963363297817978,3.2724859878416246
4422008786483604992,2.8061159372664246,0.28322948843882695,230.96828886161552,0.2202753669467863,2.776882655709402,0.20585757806277802,-10.545282269206211,0.4348205679710938,3.626420510413496,0.40995909534972347
4422017415073107328,nan,nan,231.11830942674368,0.8840346540361825,2.9373577587788855,0.9947966063839209,nan,nan,nan,nan


The system also supports Pan-STARRS1 and 2MASS cross-matches using the `panstarrs1` and `twomass` keywords

In [4]:
df = make_simple_query(
    WHERE=circle,        # The WHERE part of the SQL
    random_index=1e4,         # a shortcut to use the random_index in 'WHERE'
    ORDERBY='gaia.parallax',  # setting the data ordering
    panstarrs1=True, twomass=True,
    do_query=True,            # perform the query using gaia_tools.query
    local=False,              # whether to perform the query locally
    units=True                # to fill in missing units from 'defaults' file
)

df



starting query @ m04d02h18m25s00
Query finished.
query finished @ m04d02h18m25s01


id,prlx,prlx_err,ra,ra_err,dec,dec_err,pmra,pmra_err,pmdec,pmdec_err,g,g_err,r,r_err,i,i_err,z,z_err,j,h,k
,mas,mas,deg,deg,deg,deg,mas / yr,mas / yr,mas / yr,mas / yr,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32
4421854511258656512,1.2275574478067548,0.47971770270355213,228.8875273212023,0.47414322708214046,2.6766887835445843,0.4727195334587449,-1.086205413366316,0.8596805506822935,8.1358729037405,0.8617703124298011,20.7679004669189,0.0411490015685558,19.5503005981445,0.00723499990999699,18.4057998657227,0.00405299989506602,17.9002990722656,0.00269200000911951,16.5049991607666,15.90999984741211,15.385000228881836
4422008786483604992,2.8061159372664246,0.28322948843882695,230.96828886161552,0.2202753669467863,2.776882655709402,0.20585757806277802,-10.545282269206211,0.4348205679710938,3.626420510413496,0.40995909534972347,20.4762001037598,0.0156880002468824,19.2593002319336,0.00898800045251846,17.861400604248,0.00322599988430738,17.2089996337891,0.0047889999113977,15.788000106811523,15.204000473022461,14.838000297546387


## Different Defaults

If you want fewer default columns, this is an option through the `defaults` keyword.

In [5]:
df = make_simple_query(
    WHERE=circle, random_index=1e4, ORDERBY='gaia.parallax',
    do_query=True, local=False, units=True,
    defaults='empty', 
)

df



starting query @ m04d02h18m25s01
Query finished.
query finished @ m04d02h18m25s01


id
int64
4421854511258656512
4421967451718550400
6336481490785920000
4422008786483604992
4422017415073107328


Likewise, there's an option for much greater detail.

In [6]:
df = make_simple_query(
    WHERE=circle, random_index=1e4, ORDERBY='gaia.parallax',
    do_query=True, local=False, units=True,
    defaults='full'
)

df



starting query @ m04d02h18m25s01
Query finished.
query finished @ m04d02h18m25s02


id,ref_epoch,prlx,prlx_err,ra,ra_err,dec,dec_err,pmra,pmra_err,pmdec,pmdec_err,rvel,rvel_err,l,b,ecl_lon,ecl_lat
,yr,mas,mas,deg,deg,deg,deg,mas / yr,mas / yr,mas / yr,mas / yr,km / s,km / s,deg,deg,deg,deg
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
4421854511258656512,2015.5,1.2275574478067548,0.47971770270355213,228.8875273212023,0.47414322708214046,2.6766887835445843,0.4727195334587449,-1.086205413366316,0.8596805506822935,8.1358729037405,0.8617703124298011,nan,nan,3.848132547687008,47.75429241848735,225.65099212892665,20.01174691229745
4421967451718550400,2015.5,1.3590008052478864,0.41067570449898166,229.30428207922225,0.32917441643105966,3.504297860472648,0.30141853305113375,-15.617658856828964,0.69020127163748,-11.519501862147981,0.6119762121388062,nan,nan,5.199902605104458,47.94185070637154,225.83207814826355,20.92242336503168
6336481490785920000,2015.5,1.9709522874255738,1.4830800617599345,227.8835097013005,1.527311600754504,-3.008821326198841,1.2609050214395636,-18.116198677523734,4.053641595094551,-5.963363297817978,3.2724859878416246,nan,nan,356.7225516888186,44.73931290031377,226.28760395945173,14.270443215122377
4422008786483604992,2015.5,2.8061159372664246,0.28322948843882695,230.96828886161552,0.2202753669467863,2.776882655709402,0.20585757806277802,-10.545282269206211,0.4348205679710938,3.626420510413496,0.40995909534972347,nan,nan,5.846380077001391,46.18235257154016,227.75425415624437,20.675724379475792
4422017415073107328,2015.5,nan,nan,231.11830942674368,0.8840346540361825,2.9373577587788855,0.9947966063839209,nan,nan,nan,nan,nan,nan,6.161884327807021,46.16076695196851,227.86275770664682,20.87043033203375


<br><br><br><br>
- - -
- - -
<br><br><br><br>

# Complex Nested Query

A complex query like this one shows the real utility of this package.
Instead of keeping track of the complex SQL, we only need to pay close attention to the custom calculated columns.

This ADQL queries for data within a rectangular area on a sky rotated by a rotation matrix and specified North Galactic Pole angles. The specifics aren't important -- the real takeaway is that the sky rotation and calculation are written in a clear format, with all the parts of the query close together. Running the query is trivial after that.

In [7]:
###########
# Custom Calculations

# Innermost Level
l0cols = """
--Rotation Matrix
{K00}*cos(radians(dec))*cos(radians(ra))+
{K01}*cos(radians(dec))*sin(radians(ra))+
{K02}*sin(radians(dec)) AS cosphi1cosphi2,

{K10}*cos(radians(dec))*cos(radians(ra))+
{K11}*cos(radians(dec))*sin(radians(ra))+
{K12}*sin(radians(dec)) AS sinphi1cosphi2,

{K20}*cos(radians(dec))*cos(radians(ra))+
{K21}*cos(radians(dec))*sin(radians(ra))+
{K22}*sin(radians(dec)) AS sinphi2,

--c1, c2
{sindecngp}*cos(radians(dec)){mcosdecngp:+}*sin(radians(dec))*cos(radians(ra{mrangp:+})) as c1,
{cosdecngp}*sin(radians(ra{mrangp:+})) as c2
"""

# Inner Level
l1cols = """
gaia.cosphi1cosphi2, gaia.sinphi1cosphi2, gaia.sinphi2,
gaia.c1, gaia.c2,

atan2(sinphi1cosphi2, cosphi1cosphi2) AS phi1,
atan2(sinphi2, sinphi1cosphi2 / sin(atan2(sinphi1cosphi2, cosphi1cosphi2))) AS phi2"""

# Inner Level
l2cols = """
gaia.sinphi1cosphi2, gaia.cosphi1cosphi2, gaia.sinphi2,
gaia.phi1, gaia.phi2,
gaia.c1, gaia.c2,

( c1*pmra+c2*pmdec)/cos(phi2) AS pmphi1,
(-c2*pmra+c1*pmdec)/cos(phi2) AS pmphi2"""

# Outer Level
l3cols = """
gaia.phi1, gaia.phi2,
gaia.pmphi1, gaia.pmphi2"""

###########
# Custom Selection

l3sel = """
    phi1 > {phi1min:+}
AND phi1 < {phi1max:+}
AND phi2 > {phi2min:+}
AND phi2 < {phi2max:+}
"""

###########
# Custom substitutions

l3userasdict = {
    'K00': .656, 'K01': .755, 'K02': .002,
    'K10': .701, 'K11': .469, 'K12': .537,
    'K20': .53, 'K21': .458, 'K22': .713,
    'sindecngp': -0.925, 'cosdecngp': .382, 'mcosdecngp': -.382,
    'mrangp': -0,
    'phi1min': -0.175, 'phi1max': 0.175,
    'phi2min': -0.175, 'phi2max': 0.175}

###########
# Making Query
df = make_query(
    gaia_mags=True,
    panstarrs1=True,  # doing a Pan-STARRS1 crossmatch
    user_cols=l3cols,
    use_AS=True, user_ASdict=l3userasdict,

    # Inner Query
    FROM=make_query(
        gaia_mags=True,
        user_cols=l2cols,
    
        # Inner Query
        FROM=make_query(
            gaia_mags=True,
            user_cols=l1cols,
        
            # Innermost Query
            FROM=make_query(
                gaia_mags=True,
                inmostquery=True, # telling system this is the innermost level
                user_cols=l0cols,
                random_index=1e4  # quickly specifying random index
            )
        )
    ),
    WHERE=l3sel,
    ORDERBY='gaia.source_id',
    pprint=True,
    # doing query
    do_query=True, local=False, units=True
)

df

--Data Columns:
SELECT
--GaiaDR2 Columns:
gaia.source_id AS id,
gaia.parallax AS prlx, gaia.parallax_error AS prlx_err,
gaia.ra, gaia.ra_error AS ra_err,
gaia.dec, gaia.dec_error AS dec_err,
gaia.pmra, gaia.pmra_error AS pmra_err,
gaia.pmdec, gaia.pmdec_error AS pmdec_err,
--GaiaDR2 Magnitudes and Colors:
gaia.phot_bp_mean_mag AS Gbp,
gaia.phot_bp_mean_flux_over_error AS Gbpfluxfracerr,
gaia.phot_rp_mean_mag AS Grp,
gaia.phot_rp_mean_flux_over_error AS Grpfluxfracerr,
gaia.phot_g_mean_mag AS Gg,
gaia.phot_g_mean_flux_over_error AS Ggfluxfracerr,
gaia.bp_rp AS Gbp_rp, gaia.bp_g AS Gbp_g, gaia.g_rp AS Gg_rp,
--Adding Pan-STARRS1 Columns:
panstarrs1.g_mean_psf_mag AS g, panstarrs1.g_mean_psf_mag_error AS g_err,
panstarrs1.r_mean_psf_mag AS r, panstarrs1.r_mean_psf_mag_error AS r_err,
panstarrs1.i_mean_psf_mag AS i, panstarrs1.i_mean_psf_mag_error AS i_err,
panstarrs1.z_mean_psf_mag AS z, panstarrs1.z_mean_psf_mag_error AS z_err,

--Custom Selection & Assignement:
gaia.phi1, gaia.phi2,
gai

id,prlx,prlx_err,ra,ra_err,dec,dec_err,pmra,pmra_err,pmdec,pmdec_err,gbp,gbpfluxfracerr,grp,grpfluxfracerr,gg,ggfluxfracerr,gbp_rp,gbp_g,gg_rp,g,g_err,r,r_err,i,i_err,z,z_err,phi1,phi2,pmphi1,pmphi2
,mas,mas,deg,deg,deg,deg,mas / yr,mas / yr,mas / yr,mas / yr,mag,,mag,,mag,,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,rad,rad,mas / yr,mas / yr
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
2889564322129077376,0.3213177028021399,0.14088071404071692,89.852456363208,0.11092118907771961,-33.48788512061472,0.11905868588217561,-1.2001158428613647,0.2315703839344539,1.2068512160421232,0.2607473500694008,19.02077865600586,45.648506,17.74642562866211,108.068665,18.4609375,579.4508,1.27435302734375,0.5598411560058594,0.7145118713378906,nan,nan,nan,nan,18.1292991638184,0.00695599988102913,nan,nan,0.15177566486787875,-0.01615296804383597,1.3863772679241089,-0.4719919761842213
2889585036754358016,0.04112782798721501,0.3536404789521579,89.64273223729039,0.27550035439516946,-33.289729121056816,0.28568865747042466,3.6201024391883885,0.5441884680409558,8.104449717308881,0.5891796623791259,20.448328018188477,11.592497,18.59687614440918,50.61768,19.61026382446289,280.3479,1.8514518737792969,0.8380641937255859,1.013387680053711,nan,nan,nan,nan,18.9641990661621,0.00823699962347746,nan,nan,0.15803067899348286,-0.008958985863537603,0.30147490786345243,-7.639027705679244
2893531497522278528,0.2790765324906976,0.08843655423377134,95.69721841841746,0.06532838745037702,-30.50242870308282,0.0870745768550069,1.7096027371956273,0.14042939660904843,2.665564779495171,0.16040499468271435,17.70775032043457,156.32576,16.725849151611328,162.56195,17.290735244750977,1074.5438,0.9819011688232422,0.41701507568359375,0.5648860931396484,17.7226009368896,0.0102749997749925,17.2467002868652,0.00498000020161271,17.0774002075195,0.00384699995629489,17.0198993682861,0.00342700001783669,0.11734124245859556,-0.024506749257727892,-0.3823366779230443,-2.8264187687608446
2894277580582639616,-0.03565140127821627,0.38379135803464853,100.83588017948506,0.2746167229986943,-30.39519893204269,0.354504804621474,0.4595450174632992,0.5981503040659448,1.8037089064425966,0.6497080038713873,20.135364532470703,12.897309,19.291885375976562,18.578547,19.889904022216797,231.50648,0.8434791564941406,0.24546051025390625,0.5980186462402344,20.1900005340576,0.0153510002419353,19.7777004241943,0.0170040000230074,19.6713008880615,0.010308999568224,19.6107006072998,0.057473998516798,0.022452761935390815,-0.10979523300038331,0.29515634154001114,-1.687229325064532
2894301117005191936,-0.31822723307621215,0.596439798077345,100.55360000898901,0.37529189714934463,-30.06129384733782,0.5133368686502483,1.2125908967513244,0.7750190783329641,3.328302233757119,0.88877917171463,nan,--,nan,--,20.133920669555664,161.91643,nan,nan,nan,20.9354000091553,0.0813969969749451,19.886100769043,0.00134299998171628,19.3973999023438,0.0328940004110336,19.2562999725342,0.0201140008866787,0.0352089498571001,-0.095456026639865,0.23771429243865633,-3.251390885692647
2918472063373495936,0.07540631742850838,0.018291002548211636,101.29860970041581,0.012077191412722404,-28.707030693978805,0.016833066274518332,-0.7198628622013465,0.024939448508541573,1.3504572377243913,0.031049207460582387,14.951737403869629,588.6317,13.43400764465332,949.3284,14.241029739379883,4129.2656,1.5177297592163086,0.7107076644897461,0.8070220947265625,15.145299911499,0.00218899990431964,14.2363004684448,0.00301899993792176,13.8430995941162,0.00734200002625585,13.6037998199463,0.00123199995141476,0.0466047869519749,-0.07376865860472206,1.1188268142430042,-0.8769080774380797
2918550644092742272,0.6273189056681521,0.782697310231922,101.576417046875,0.4883607934153599,-28.75376229315029,0.6028598697415664,2.6738991917543244,1.0275467338132094,6.6712988587220